In [1]:
pwd = !pwd
pwd = pwd[0]

config_path = '/'.join(pwd.split('/')[:-1]) + '/examples/configs/normalization/fit/default.yaml'

EVAL_MAN_PATHS = {
  'CoQA': pwd + '/polygraph_tacl_stablelm12b_coqa.man',
  'GSM8K': pwd + '/polygraph_tacl_stablelm12b_gsm8k.man',
  'MMLU': pwd + '/polygraph_tacl_stablelm12b_mmlu.man',
  'TriviaQA': pwd + '/polygraph_tacl_stablelm12b_triviaqa.man',
  'WMT14': pwd + '/polygraph_tacl_stablelm12b_wmt14.man',
  'WMT19': pwd + '/polygraph_tacl_stablelm12b_wmt19.man',
  'XSum': pwd + '/polygraph_tacl_stablelm12b_xsum.man',
}

TRAIN_MAN_PATHS = {
  'CoQA': pwd + '/polygraph_tacl_stablelm12b_coqa_train.man',
  'GSM8K': pwd + '/polygraph_tacl_stablelm12b_gsm8k_train.man',
  'MMLU': pwd + '/polygraph_tacl_stablelm12b_mmlu_train.man',
  'TriviaQA': pwd + '/polygraph_tacl_stablelm12b_triviaqa_train.man',
  'WMT14': pwd + '/polygraph_tacl_stablelm12b_wmt14_train.man',
  'WMT19': pwd + '/polygraph_tacl_stablelm12b_wmt19_train.man',
  'XSum': pwd + '/polygraph_tacl_stablelm12b_xsum_train.man',
}

DATASET_NAMES = list(TRAIN_MAN_PATHS.keys())

UE_METHOD_NAMES = [
   'MaximumSequenceProbability',
   'Perplexity',
   'MeanTokenEntropy',
   'MeanPointwiseMutualInformation',
   'MeanConditionalPointwiseMutualInformation',
   'PTrue',
   'PTrueSampling',
   'MonteCarloSequenceEntropy',
   'MonteCarloNormalizedSequenceEntropy',
   'LexicalSimilarity_rouge1',
   'LexicalSimilarity_rouge2',
   'LexicalSimilarity_rougeL',
   'LexicalSimilarity_BLEU',
   'NumSemSets',
   'EigValLaplacian_NLI_score_entail',
   'EigValLaplacian_NLI_score_contra',
   'EigValLaplacian_Jaccard_score',
   'DegMat_NLI_score_entail',
   'DegMat_NLI_score_contra',
   'DegMat_Jaccard_score',
   'Eccentricity_NLI_score_entail',
   'Eccentricity_NLI_score_contra',
   'Eccentricity_Jaccard_score',
   'SemanticEntropy',
]

GEN_METRIC_NAMES = ['AlignScore']

In [2]:
!wget -r --cut-dirs=2 -nH --no-parent -A '*man' http://209.38.249.180:8000/polygraph_data/mans/

--2024-05-17 15:20:39--  http://209.38.249.180:8000/polygraph_data/mans/
Connecting to 209.38.249.180:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3165 (3.1K) [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp      100%[===================>]   3.09K  --.-KB/s    in 0s      

2024-05-17 15:20:40 (24.7 MB/s) - ‘index.html.tmp’ saved [3165/3165]

Loading robots.txt; please ignore errors.
--2024-05-17 15:20:40--  http://209.38.249.180:8000/robots.txt
Connecting to 209.38.249.180:8000... connected.
HTTP request sent, awaiting response... 404 File not found
2024-05-17 15:20:40 ERROR 404: File not found.

Removing index.html.tmp since it should be rejected.

--2024-05-17 15:20:40--  http://209.38.249.180:8000/polygraph_data/mans/polygraph_tacl_mistral7b_coqa.man
Connecting to 209.38.249.180:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5449156 (5.2M) [application/x-troff-man]
Saving to: ‘polygraph_tacl_mistral7b_coqa.man’

polygrap

In [3]:
def get_man_paths_list(man_paths):
    paths = ['"' + path + '"' for path in man_paths]
    paths = '\'[' + ','.join(paths) + ']\''

    return paths

In [4]:
import os

train_man_paths = get_man_paths_list(list(TRAIN_MAN_PATHS.values()))
os.system(f'HYDRA_CONFIG={config_path} polygraph_normalize save_path="./" man_paths={train_man_paths}')

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2954ef970>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x295505de0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x295506770>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x100b6a680>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x29543b220>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x295438ac0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2954b01c0>]


0

In [5]:
import pickle

with open('fitted_normalizers.json', 'rb') as f:
    fitted_normalizers = pickle.load(f)

In [6]:
import numpy as np
from collections import defaultdict
from lm_polygraph.normalizers.minmax import MinMaxNormalizer
from lm_polygraph.normalizers.quantile import QuantileNormalizer
from lm_polygraph.normalizers.binned_pcc import BinnedPCCNormalizer
from lm_polygraph.normalizers.isotonic_pcc import IsotonicPCCNormalizer

NORMALIZERS = {
    'min_max': MinMaxNormalizer,
    'quantile': QuantileNormalizer,
    'binned_pcc': BinnedPCCNormalizer,
    'isotonic_pcc': IsotonicPCCNormalizer
}

def get_confidences(normalizers, ues):
    confidences = {'min_max': defaultdict(dict),
                   'quantile': defaultdict(dict),
                   'binned_pcc': defaultdict(dict),
                   'isotonic_pcc': defaultdict(dict)}
    
    for key in confidences.keys():
        for method_name in UE_METHOD_NAMES:
            for metric_name in GEN_METRIC_NAMES:
                normalizer = NORMALIZERS[key].loads(normalizers[(metric_name, method_name, key)])
                confidences[key][metric_name][method_name] = normalizer.transform(ues[method_name])

    return confidences

def calculate_mses(confidences, gen_metrics):
    mses = {'min_max': defaultdict(dict),
            'quantile': defaultdict(dict),
            'binned_pcc': defaultdict(dict),
            'isotonic_pcc': defaultdict(dict)}
    
    for key in confidences.keys():
        for metric_name in GEN_METRIC_NAMES:
            metric_mses = []
            for method_name in UE_METHOD_NAMES:
                gt_metric = gen_metrics[metric_name]
                mse = ((confidences[key][metric_name][method_name] - gt_metric) ** 2).mean()
                metric_mses.append(mse)
            mses[key][metric_name] = metric_mses

    return mses

/opt/homebrew/Caskroom/miniconda/base/envs/ue_abssum/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### All datasets

In [7]:
from lm_polygraph.utils.normalize import get_mans_ues_metrics

ues, gen_metrics = get_mans_ues_metrics(EVAL_MAN_PATHS.values(), UE_METHOD_NAMES, GEN_METRIC_NAMES)

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x105ff3be0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a1bac0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a385b0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a38be0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a39210>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a397e0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a5a39e10>]


In [8]:
confidences = get_confidences(fitted_normalizers, ues)
mses = calculate_mses(confidences, gen_metrics)

In [9]:
def plot_mses(ax, mses, title):
    ax.bar(x-0.3, mses['min_max']['AlignScore'], width=0.2, color='g', align='center', label='MinMax')
    ax.bar(x-0.1, mses['quantile']['AlignScore'], width=0.2, color='b', align='center', label='Quantile')
    ax.bar(x+0.1, mses['binned_pcc']['AlignScore'], width=0.2, color='tab:olive', align='center', label='Binned')
    ax.bar(x+0.3, mses['isotonic_pcc']['AlignScore'], width=0.2, color='r', align='center', label='Isotonic')

    ax.set_xticks(range(len(UE_METHOD_NAMES)), UE_METHOD_NAMES, rotation=90)
    
    ax.set_title(title)
    ax.set_ylabel('MSE')

In [10]:
import matplotlib.pyplot as plt

x = np.array(list(range(len(UE_METHOD_NAMES))))

f, ax = plt.subplots(1, 1, figsize=(9, 9))

plot_mses(ax, mses, 'MSE between true AlignScore and confidence')

handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, bbox_to_anchor=(1.13, 0.97))

plt.tight_layout()
plt.savefig(f'normalization_mse_total.pdf')
plt.clf()

<Figure size 900x900 with 0 Axes>

### OOD Datasets

In [11]:
ood_confidences = {}
ood_mses = {}

for dataset_name in DATASET_NAMES:
    # Fit normalizers excluding current OOD dataset from train set
    train_man_paths_wo_dataset = [value for key, value in TRAIN_MAN_PATHS.items() if key != dataset_name]
    train_man_paths_wo_dataset = get_man_paths_list(train_man_paths_wo_dataset)
    os.system(f'HYDRA_CONFIG={config_path} polygraph_normalize save_path="./ood_{dataset_name}" man_paths={train_man_paths_wo_dataset}')

    # Get UE and metric values for OOD dataset
    ues, gen_metrics = get_mans_ues_metrics([EVAL_MAN_PATHS[dataset_name]], UE_METHOD_NAMES, GEN_METRIC_NAMES)

    with open(f'./ood_{dataset_name}/fitted_normalizers.json', 'rb') as f:
        fitted_normalizers = pickle.load(f)

    ood_confidences[dataset_name] = get_confidences(fitted_normalizers, ues)
    ood_mses[dataset_name] = calculate_mses(ood_confidences[dataset_name], gen_metrics)

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2994e7d90>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2994f9ff0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x10544db10>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x29926ea40>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x29926c3a0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x29926d4e0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a7413d60>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a14e7a90>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2a14fa020>]
Stat calculators: [

In [12]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

is_even = (len(DATASET_NAMES) % 2 == 0)
even_datasets_subset = DATASET_NAMES if is_even else DATASET_NAMES[:-1]
dataset_pairs = [even_datasets_subset[i:i + 2] for i in range(0, len(even_datasets_subset), 2)]

x = np.array(list(range(len(UE_METHOD_NAMES))))

for dataset_pair in dataset_pairs:
    fig = plt.figure(figsize=(12, 6))
    gs = GridSpec(1, 2, figure=fig)

    for i, dataset_name in enumerate(dataset_pair):
        ax = fig.add_subplot(gs[i//2,i%2])
        plot_mses(ax, ood_mses[dataset_name], f'MSE between true AlignScore and confidence: {dataset_name}')
        handles, labels = ax.get_legend_handles_labels()

    fig.legend(handles, labels, bbox_to_anchor=(1.1, 0.95))
    
    plt.tight_layout()
    plt.savefig(f'normalization_mse_ood_{dataset_name}.pdf')
    plt.clf()

if not is_even:
    dataset_name = DATASET_NAMES[-1]
    
    fig = plt.figure(figsize=(6, 6))
    gs = GridSpec(1, 1, figure=fig)

    ax = fig.add_subplot(gs[0,0])

    plot_mses(ax, ood_mses[dataset_name], f'MSE between true AlignScore and confidence: {dataset_name}')
    
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, bbox_to_anchor=(1.2, 0.95))
    
    plt.tight_layout()
    plt.savefig(f'normalization_mse_ood_{dataset_name}.pdf')
    plt.clf()

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 600x600 with 0 Axes>

### PRR change relative to raw uncertainty

In [13]:
from lm_polygraph.utils.normalize import filter_nans
from lm_polygraph.ue_metrics.pred_rej_area import PredictionRejectionArea
from lm_polygraph.ue_metrics.ue_metric import (
    get_random_scores,
    normalize_metric,
)
import pandas as pd
from IPython.core.debugger import set_trace

ue_metric = PredictionRejectionArea()

cols = ['MinMax', 'Quantile', 'Binned PCC', 'Isotonic PCC']

for dataset_name, path in EVAL_MAN_PATHS.items():
    res = {}
    all_ues, all_gen_metrics = get_mans_ues_metrics([path], UE_METHOD_NAMES, GEN_METRIC_NAMES)
    
    for metric_name in GEN_METRIC_NAMES:
        gen_metrics = all_gen_metrics[metric_name]
        for ue_method_name in UE_METHOD_NAMES:
            ues = all_ues[ue_method_name]
            
            filtered_metric, filtered_ues = filter_nans(gen_metrics, ues)
            
            minmax_ues = -np.array(ood_confidences[dataset_name]['min_max'][metric_name][ue_method_name])
            quantile_ues = -np.array(ood_confidences[dataset_name]['quantile'][metric_name][ue_method_name])
            binned_pcc_ues = -np.array(ood_confidences[dataset_name]['binned_pcc'][metric_name][ue_method_name])
            isotonic_pcc_ues = -np.array(ood_confidences[dataset_name]['isotonic_pcc'][metric_name][ue_method_name])

            assert(len(filtered_ues) == len(minmax_ues))
            
            oracle_score = ue_metric(-filtered_metric, filtered_metric)
            random_score = get_random_scores(ue_metric, filtered_metric)

            raw_ue_metric_val = ue_metric(filtered_ues, filtered_metric)
            raw_score = normalize_metric(raw_ue_metric_val, oracle_score, random_score)

            minmax_ue_metric_val = ue_metric(minmax_ues, filtered_metric)
            minmax_score = normalize_metric(minmax_ue_metric_val, oracle_score, random_score)
            minmax_diff = raw_score - minmax_score

            quantile_ue_metric_val = ue_metric(quantile_ues, filtered_metric)
            quantile_score = normalize_metric(quantile_ue_metric_val, oracle_score, random_score)
            quantile_diff = raw_score - quantile_score
            
            binned_pcc_ue_metric_val = ue_metric(binned_pcc_ues, filtered_metric)
            binned_pcc_score = normalize_metric(binned_pcc_ue_metric_val, oracle_score, random_score)
            binned_pcc_diff = raw_score - binned_pcc_score

            isotonic_pcc_ue_metric_val = ue_metric(isotonic_pcc_ues, filtered_metric)
            isotonic_pcc_score = normalize_metric(isotonic_pcc_ue_metric_val, oracle_score, random_score)
            isotonic_pcc_diff = raw_score - isotonic_pcc_score

            res[ue_method_name] = [minmax_diff, quantile_diff, binned_pcc_diff, isotonic_pcc_diff]

    df = pd.DataFrame.from_dict(res, orient='index', columns=cols)
    display(df)

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x105aff610>]


,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,0.000000,0.000000e+00,0.045624,0.000000e+00
Perplexity,0.000054,5.432411e-05,0.017120,1.025691e-04
MeanTokenEntropy,0.000000,0.000000e+00,0.071875,1.270313e-04
MeanPointwiseMutualInformation,0.000000,0.000000e+00,0.023563,1.318007e-06
MeanConditionalPointwiseMutualInformation,0.000054,5.432411e-05,0.017120,1.025691e-04
PTrue,0.000000,0.000000e+00,-0.040076,-1.698985e-02
PTrueSampling,0.000000,0.000000e+00,-0.005526,9.455222e-02
MonteCarloSequenceEntropy,0.000000,0.000000e+00,0.072111,3.595616e-04
MonteCarloNormalizedSequenceEntropy,0.000043,4.294049e-05,0.026097,2.124271e-04
LexicalSimilarity_rouge1,0.000000,5.962999e-05,0.240101,4.681824e-04


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aa3fb040>]


,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,0.000000,0.000000,-0.014490,0.000000e+00
Perplexity,0.000000,0.000000,0.077725,0.000000e+00
MeanTokenEntropy,0.000000,0.000000,0.051275,0.000000e+00
MeanPointwiseMutualInformation,0.000000,0.000000,0.372224,0.000000e+00
MeanConditionalPointwiseMutualInformation,0.000000,0.000000,0.077725,0.000000e+00
PTrue,0.000000,0.000000,0.014158,0.000000e+00
PTrueSampling,0.000000,0.000000,0.059528,0.000000e+00
MonteCarloSequenceEntropy,0.000199,0.000199,0.100969,5.841522e-03
MonteCarloNormalizedSequenceEntropy,0.000000,0.000000,0.026921,0.000000e+00
LexicalSimilarity_rouge1,0.000000,0.000000,0.017079,0.000000e+00


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2ab54f700>]


...

,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,0.000000,0.000000,0.146019,0.000000e+00
Perplexity,0.000000,0.000000,0.023779,0.000000e+00
MeanTokenEntropy,0.000000,0.000000,0.040141,0.000000e+00
MeanPointwiseMutualInformation,0.000000,0.000000,0.015287,0.000000e+00
MeanConditionalPointwiseMutualInformation,0.000000,0.000000,0.023779,0.000000e+00
PTrue,0.000000,0.000000,0.124073,3.356028e-02
PTrueSampling,0.000000,0.000000,0.135484,1.691108e-02
MonteCarloSequenceEntropy,0.000000,0.000000,0.202491,0.000000e+00
MonteCarloNormalizedSequenceEntropy,0.000000,0.000000,-0.028530,0.000000e+00
LexicalSimilarity_rouge1,0.000000,-0.000143,0.054083,9.814974e-04


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x105b215d0>]


,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,0.000000,0.000000e+00,0.093635,0.000000e+00
Perplexity,0.000000,0.000000e+00,0.004554,0.000000e+00
MeanTokenEntropy,0.000000,0.000000e+00,0.016244,0.000000e+00
MeanPointwiseMutualInformation,0.000000,0.000000e+00,-0.067971,0.000000e+00
MeanConditionalPointwiseMutualInformation,0.000000,0.000000e+00,0.004554,0.000000e+00
PTrue,0.000000,0.000000e+00,0.270316,4.404489e-04
PTrueSampling,0.000000,0.000000e+00,0.054360,2.749514e-04
MonteCarloSequenceEntropy,0.000000,0.000000e+00,0.233209,0.000000e+00
MonteCarloNormalizedSequenceEntropy,0.000000,0.000000e+00,-0.015257,0.000000e+00
LexicalSimilarity_rouge1,0.000000,2.528180e-07,0.042520,3.040329e-05


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x105f6e9b0>]


,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,0.0,0.0,-0.021840,0.000000e+00
Perplexity,0.0,0.0,0.011166,0.000000e+00
MeanTokenEntropy,0.0,0.0,0.019600,0.000000e+00
MeanPointwiseMutualInformation,0.0,0.0,0.045119,0.000000e+00
MeanConditionalPointwiseMutualInformation,0.0,0.0,0.011166,0.000000e+00
PTrue,0.0,0.0,-0.050726,3.519649e-04
PTrueSampling,0.0,0.0,-0.017532,1.190431e-03
MonteCarloSequenceEntropy,0.0,0.0,-0.014168,0.000000e+00
MonteCarloNormalizedSequenceEntropy,0.0,0.0,-0.000438,0.000000e+00
LexicalSimilarity_rouge1,0.0,0.0,0.083852,-5.900041e-05


In [14]:
all_ues, all_gen_metrics = get_mans_ues_metrics(list(EVAL_MAN_PATHS.values()), UE_METHOD_NAMES, GEN_METRIC_NAMES)

for metric_name in GEN_METRIC_NAMES:
    gen_metrics = all_gen_metrics[metric_name]
    for ue_method_name in UE_METHOD_NAMES:
        ues = all_ues[ue_method_name]
        
        filtered_metric, filtered_ues = filter_nans(gen_metrics, ues)
        
        minmax_ues = -np.array(confidences['min_max'][metric_name][ue_method_name])
        quantile_ues = -np.array(confidences['quantile'][metric_name][ue_method_name])
        binned_pcc_ues = -np.array(confidences['binned_pcc'][metric_name][ue_method_name])
        isotonic_pcc_ues = -np.array(confidences['isotonic_pcc'][metric_name][ue_method_name])

        assert(len(filtered_ues) == len(minmax_ues))
        
        oracle_score = ue_metric(-filtered_metric, filtered_metric)
        random_score = get_random_scores(ue_metric, filtered_metric)

        raw_ue_metric_val = ue_metric(filtered_ues, filtered_metric)
        raw_score = normalize_metric(raw_ue_metric_val, oracle_score, random_score)

        minmax_ue_metric_val = ue_metric(minmax_ues, filtered_metric)
        minmax_score = normalize_metric(minmax_ue_metric_val, oracle_score, random_score)
        minmax_diff = raw_score - minmax_score

        quantile_ue_metric_val = ue_metric(quantile_ues, filtered_metric)
        quantile_score = normalize_metric(quantile_ue_metric_val, oracle_score, random_score)
        quantile_diff = raw_score - quantile_score
        
        binned_pcc_ue_metric_val = ue_metric(binned_pcc_ues, filtered_metric)
        binned_pcc_score = normalize_metric(binned_pcc_ue_metric_val, oracle_score, random_score)
        binned_pcc_diff = raw_score - binned_pcc_score

        isotonic_pcc_ue_metric_val = ue_metric(isotonic_pcc_ues, filtered_metric)
        isotonic_pcc_score = normalize_metric(isotonic_pcc_ue_metric_val, oracle_score, random_score)
        isotonic_pcc_diff = raw_score - isotonic_pcc_score

        res[ue_method_name] = [minmax_diff, quantile_diff, binned_pcc_diff, isotonic_pcc_diff]

df = pd.DataFrame.from_dict(res, orient='index', columns=cols)
display(df)

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x105f6ef80>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aaebf520>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aaebe320>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aaebf760>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aaebcb20>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aaebeef0>]
Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x2aae12500>]


,MinMax,Quantile,Binned PCC,Isotonic PCC
MaximumSequenceProbability,-5.721821e-06,-5.721821e-06,-0.050397,-5.721821e-06
Perplexity,-8.007145e-06,-1.108380e-05,0.018182,-7.831774e-06
MeanTokenEntropy,3.145503e-10,3.145503e-10,-0.008805,3.145503e-10
MeanPointwiseMutualInformation,0.000000e+00,0.000000e+00,-0.313686,2.883105e-08
MeanConditionalPointwiseMutualInformation,-8.007145e-06,-8.007145e-06,0.018182,-7.831774e-06
PTrue,-3.504455e-05,-3.504455e-05,-0.759635,-3.782925e-01
PTrueSampling,0.000000e+00,0.000000e+00,-0.375160,-5.205038e-02
MonteCarloSequenceEntropy,-5.044093e-06,-5.044093e-06,-0.068054,-5.044093e-06
MonteCarloNormalizedSequenceEntropy,3.145507e-10,3.145507e-10,0.009059,3.145507e-10
LexicalSimilarity_rouge1,0.000000e+00,4.949279e-06,0.062659,-1.029108e-05
